<a href="https://colab.research.google.com/github/1Englert1/cerf_classification/blob/main/dev/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

## Load Data

In [3]:
%%capture
!pip install tqdm
from tqdm.notebook import tqdm
tqdm().pandas()
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/1Englert1/cerf_classification/main/dev/data/texts_sm.csv")

#Trim Strings
for index in tqdm(df.index):
    df.at[index, "Text File"] = df["Text File"][index].strip()





In [4]:
df.head()

,Text File,lvl
0,sehr gefruet auf diser Platz. Deshalb möchte i...,B1
1,"Sehr geehrte Damen und Herren,mit grossem Inte...",B2
2,Sehr geehrte Damen und Herrenhiermit habe ich ...,B1
3,"Für Leute auf der Welt ist es schon eindeutig,...",B2
4,Ich bin der Meinung das man sollte sich an der...,B2


## Fehler auslesen u. korregieren (language Tool)
https://pypi.org/project/language-tool-python/

In [ ]:
#%%capture
!pip install language-tool-python

In [ ]:
import language_tool_python
tool = language_tool_python.LanguageTool('de-DE')  

Unzipping /tmp/tmpdp4bdwxp.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.1.zip to /root/.cache/language_tool_python.


In [ ]:
df[["er_wspace","er_untr","er_uncat","er_typ","er_term","er_style","er_reg","er_pattern","er_other","er_omis","er_num","er_noncon","er_mistrans","er_misssp","er_mark","er_locvio","er_locspec","er_len","er_leg","er_intern","er_intit","er_inste","er_gram","er_dup","er_wspace","er_chars","er_add"]] = 0
for index in tqdm(df.index):
  text = df.at[index, "Text File"]
  df.at[index, "cor_text"] = tool.correct(text)
  matches = tool.check(text)
  df.at[index, "er_total"] = len(matches) 
  i = 0
  for match in matches:
    error = matches[i]
    i += 1
    if error.ruleIssueType == "whitespace":
     df.at[index, "er_wspace"] += 1
    if error.ruleIssueType == "untranslated":
     df.at[index, "er_untr"] += 1
    if error.ruleIssueType == "uncategorized":
     df.at[index, "er_uncat"] += 1
    if error.ruleIssueType == "typographical":
     df.at[index, "er_typ"] += 1
    if error.ruleIssueType == "terminology":
     df.at[index, "er_term"] += 1
    if error.ruleIssueType == "style":
     df.at[index, "er_style"] += 1
    if error.ruleIssueType == "register":
     df.at[index, "er_reg"] += 1
    if error.ruleIssueType == "patternproblem":
     df.at[index, "er_pattern"] += 1
    if error.ruleIssueType == "other":
     df.at[index, "er_other"] += 1
    if error.ruleIssueType == "omission":
     df.at[index, "er_omis"] += 1
    if error.ruleIssueType == "numbers":
     df.at[index, "er_num"] += 1
    if error.ruleIssueType == "nonconformance":
     df.at[index, "er_noncon"] += 1
    if error.ruleIssueType == "mistranslation":
     df.at[index, "er_mistrans"] += 1
    if error.ruleIssueType == "misspelling":
     df.at[index, "er_misssp"] += 1
    if error.ruleIssueType == "markup":
     df.at[index, "er_mark"] += 1
    if error.ruleIssueType == "localeviolation":
     df.at[index, "er_locvio"] += 1
    if error.ruleIssueType == "localespecificcontent":
     df.at[index, "er_locspec"] += 1
    if error.ruleIssueType == "length":
     df.at[index, "er_len"] += 1
    if error.ruleIssueType == "legal":
     df.at[index, "er_leg"] += 1
    if error.ruleIssueType == "internationalization":
     df.at[index, "er_intern"] += 1
    if error.ruleIssueType == "inconsistententities":
     df.at[index, "er_intit"] += 1
    if error.ruleIssueType == "inconsistency":
     df.at[index, "er_inste"] += 1
    if error.ruleIssueType == "grammar":
     df.at[index, "er_gram"] += 1
    if error.ruleIssueType == "duplication":
     df.at[index, "er_dup"] += 1
    if error.ruleIssueType == "whitespace":
     df.at[index, "er_wspace"] += 1
    if error.ruleIssueType == "characters":
     df.at[index, "er_chars"] += 1
    if error.ruleIssueType == "addition":
     df.at[index, "er_add"] += 1
    
    
print(df[["er_wspace","er_untr","er_uncat","er_typ","er_term","er_style","er_reg","er_pattern","er_other","er_omis","er_num","er_noncon","er_mistrans","er_misssp","er_mark","er_locvio","er_locspec","er_len","er_leg","er_intern","er_intit","er_inste","er_gram","er_dup","er_wspace","er_chars","er_add"]])


   er_wspace  er_untr  er_uncat  er_typ  ...  er_dup  er_wspace  er_chars  er_add
0          4        0         6       1  ...       0          4         0       0
1          2        0         1       0  ...       0          2         0       0
2          0        0         5       1  ...       0          0         0       0
3          6        0         2       0  ...       1          6         0       0
4         10        0        22       0  ...       0         10         0       0
5          0        0         3       0  ...       0          0         0       0
6          6        0         1       1  ...       0          6         0       0
7          6        0         5       0  ...       0          6         0       0

[8 rows x 27 columns]


## Oberflächliche Features

In [ ]:
# Tool: Taxtacy
#https://chartbeat-labs.github.io/textacy/build/html/getting_started/quickstart.html
%%capture
!pip install textacy


In [ ]:
!python -m spacy download de_core_news_sm
!python -m spacy link de_core_news_sm de

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
import textacy
import string 
de = spacy.load('de')  
#de = textacy.load_spacy_lang("de_core_news_sm")
#de = de_core_news_sm.load()

#basic counts
for index in tqdm(df.index):
  doc = textacy.make_spacy_doc(df.at[index, "cor_text"], lang=de)
  ts = textacy.text_stats.TextStats(doc)
  df.at[index, "sents_count"] = ts.n_sents
  df.at[index, "word_count"] = ts.n_words
  df.at[index, "char_count"] = ts.n_chars
  df.at[index, "avr_sents_leng"] = df["word_count"][index] / df["sents_count"][index]
  df.at[index, "avr_char_p_word"] = df["char_count"][index] / df["word_count"][index]
  df.at[index, "avr_syllables_p_word"] =  ts.n_syllables / df["word_count"][index]

## Syntaktische Features

In [ ]:
!pip install Stanza
import stanza
stanza.install_corenlp(dir="/content/corenlp")
import os
os.environ["CORENLP_HOME"] = "./corenlp"
stanza.download_corenlp_models(model='german', version='4.1.0', dir="/content/corenlp")

     |████████████████████████████████| 235kB 2.8MB/s 


Installing CoreNLP package into /content/corenlp...
For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=/content/corenlp`.


In [ ]:
from stanza.server import CoreNLPClient

In [ ]:
client = CoreNLPClient(properties='german',
                   annotators='tokenize,ssplit,mwt,pos,parse',
                   timeout=300000,
                   output_format='text', 
                   be_quiet=True, 
                   endpoint='http://localhost:9001',
                   memory='8G')


Using Stanford CoreNLP default properties for: german.  Make sure to have german models jar (available for download here: https://stanfordnlp.github.io/CoreNLP/) in CLASSPATH


In [ ]:
client.start()
import time; time.sleep(10)

Starting server with command: java -Xmx8G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 300000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties german -annotators tokenize,ssplit,mwt,pos,parse -preload -outputFormat text


In [ ]:
def match_pattern(_pattern, _text):
  matches = client.tregex(_text,_pattern)
  list = matches['sentences']
  for sentence in matches['sentences']:
    for match_id in sentence:
      print(sentence[match_id]['spanString'])

def count_matches(_pattern, _text):
  matches = client.tregex(_text,_pattern)
  list = matches['sentences']
  count_units = 0
  for sentence in matches['sentences']:
    for match_id in sentence:
      count_units = count_units + 1
  return(count_units)

def len_sum_matches(_pattern, _text):
  matches = client.tregex(_text,_pattern)
  list = matches['sentences']
  match_list = []
  for sentence in matches['sentences']:
    for match_id in sentence:
      match_list.append(sentence[match_id]['spanString'])
  len_sum = 0
  for item in match_list:
    doc = textacy.make_spacy_doc(item, lang=de)
    ts = textacy.text_stats.TextStats(doc)
    len_sum = len_sum + ts.n_words
  return(len_sum)

In [ ]:
for index in tqdm(df.index):
  text = df.at[index, "cor_text"]
  ann = client.annotate(text)

################T-Units#####################
  pattern = "(S > NUR) | (S >> CS >> NUR)"
  df.at[index, "TUnit"] = count_matches(pattern, text)

################Dep. Clause#####################
  pattern = "S >> (S > (CS > NUR > ROOT) | > (NUR > ROOT))"
  df.at[index, "dep_clause"] = count_matches(pattern, text)

################Dep. Clause with conjunction#####################
  pattern = "S < SCONJ"
  df.at[index, "dep_clause_w_conj"] = count_matches(pattern, text)
################non-Terminals#####################
  pattern = "VZ"
  df.at[index, "non_term"] = count_matches(pattern, text)

################zu-marked infinitive#####################
  pattern = "AA | AP | AVP | CAC | CAP | CAVP | CCP | CH | CNP | CO | CPP | CS | CVP | CVZ | DL | ISU | MPN | MTA | NM | NP | PP | QL | S | VP | VZ"
  df.at[index, "vz"] = count_matches(pattern, text)

################Avg. length of VP#####################
  pattern = "VP"
  df.at[index, "vp_count"] = count_matches(pattern, text)
  df.at[index, "vp_sum_len"] = len_sum_matches(pattern, text)


In [ ]:
df["len_TUnit"] = df["TUnit"] / df["word_count"]
df["len_dep_clause"] = df["dep_clause"] / df["word_count"]
df["dep_clause_w_conj_to_dep_clause"] = df["dep_clause_w_conj"] / df["dep_clause"]
df["non_term_to_words"] = df["non_term"] / df["word_count"]
df["vz_to_sents"] = df["vz"] / df["sents_count"]
df["len_vp_to_vp"] = df["vp_sum_len"] / df["vp_count"]
print("--------DONE---------")

--------DONE---------


In [ ]:
print(df[["len_vp_to_vp", "vp_sum_len", "vp_count"]])

   len_vp_to_vp  vp_sum_len  vp_count
0      3.571429        25.0       7.0
1      5.000000        25.0       5.0
2      5.142857        36.0       7.0
3      6.800000       136.0      20.0
4      4.700000        47.0      10.0
5      4.000000        12.0       3.0
6           NaN         0.0       0.0
7      5.200000        26.0       5.0


In [ ]:
client.stop()

time.sleep(10)
!ps -o pid,cmd | grep java